# Análisis de Características de Audio - Fake vs Original

# A. Base de datos


### A.1. Descripción


### A.2. Procesamiento de audios indivituales

# B. Descripción de las características obtenidas de cada audio

Explicar fundamentos de las características obtenidas con estas funciones
```
extract_temporal_features

extract_envelope_features

extract_spectral_features

extract_harmonic_features

extract_mfcc_features

extract_mel_spectrogram_features
```

## 1. Importar Librerías Necesarias

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import Audio, display
import os
import glob
import warnings
warnings.filterwarnings('ignore')

## 2. Leer el Dataset de Características de Audio

In [5]:
csv_path = 'results/fake_audio_features_20250929_092106.csv'
df = pd.read_csv(csv_path)

print(f"Dimensiones del dataset: {df.shape}")
print(f"Columnas: {list(df.columns)}")

Dimensiones del dataset: (360, 206)
Columnas: ['filename', 'original_duration', 'processed_duration', 'sample_rate', 'duration', 'samples', 'mean_amplitude', 'std_amplitude', 'max_amplitude', 'rms', 'skewness', 'kurtosis', 'zero_crossing_rate', 'energy_mean', 'energy_std', 'energy_max', 'energy_min', 'energy_entropy', 'attack_time', 'decay_time', 'sustain_level', 'release_time', 'envelope_mean', 'envelope_std', 'envelope_max', 'envelope_range', 'spectral_centroid_mean', 'spectral_centroid_std', 'spectral_rolloff_mean', 'spectral_rolloff_std', 'spectral_bandwidth_mean', 'spectral_bandwidth_std', 'spectral_contrast_0_mean', 'spectral_contrast_0_std', 'spectral_contrast_1_mean', 'spectral_contrast_1_std', 'spectral_contrast_2_mean', 'spectral_contrast_2_std', 'spectral_contrast_3_mean', 'spectral_contrast_3_std', 'spectral_contrast_4_mean', 'spectral_contrast_4_std', 'spectral_contrast_5_mean', 'spectral_contrast_5_std', 'spectral_contrast_6_mean', 'spectral_contrast_6_std', 'spectral_fla

### 2.1 Mostrar las primeras filas del dataset

In [6]:
# 1.1 Mostrar las primeras filas con pandas
print("Primeras 5 filas del dataset:")
display(df.head())

print("\nInformación del dataset:")
print(df.info())

print("\nEstadísticas descriptivas:")
display(df.describe())

Primeras 5 filas del dataset:


,filename,original_duration,processed_duration,sample_rate,duration,samples,mean_amplitude,std_amplitude,max_amplitude,rms,...,mel_band_4_std,mel_band_5_mean,mel_band_5_std,mel_band_6_mean,mel_band_6_std,mel_band_7_mean,mel_band_7_std,individual_id,audio_type,label
0,data/procesados/1701/original/1701_original_02...,5.0,5.0,22050,5.0,110250,0.120633,0.181460,1.0,0.181460,...,8.887956,-58.541992,8.131583,-67.552574,8.603199,-79.842865,1.379036,1701,original,0
1,data/procesados/1701/original/1701_original_03...,5.0,5.0,22050,5.0,110250,0.128366,0.182944,1.0,0.182944,...,8.699811,-54.619404,8.131659,-61.560776,9.732371,-79.977510,0.329748,1701,original,0
2,data/procesados/1701/original/1701_original_03...,5.0,5.0,22050,5.0,110250,0.126642,0.191174,1.0,0.191174,...,7.860029,-57.558060,8.827160,-65.743904,10.473632,-79.998860,0.016779,1701,original,0
3,data/procesados/1701/original/1701_original_02...,5.0,5.0,22050,5.0,110250,0.160985,0.221658,1.0,0.221658,...,8.653882,-54.521553,7.845850,-61.524430,9.324145,-79.872620,1.349337,1701,original,0
4,data/procesados/1701/original/1701_original_03...,5.0,5.0,22050,5.0,110250,0.130749,0.193235,1.0,0.193235,...,9.576226,-53.725727,8.191900,-64.833150,8.528844,-79.956790,0.533269,1701,original,0



Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Columns: 206 entries, filename to label
dtypes: float64(200), int64(4), object(2)
memory usage: 579.5+ KB
None

Estadísticas descriptivas:


,original_duration,processed_duration,sample_rate,duration,samples,mean_amplitude,std_amplitude,max_amplitude,rms,skewness,...,mel_band_4_mean,mel_band_4_std,mel_band_5_mean,mel_band_5_std,mel_band_6_mean,mel_band_6_std,mel_band_7_mean,mel_band_7_std,individual_id,label
count,360.0,360.0,360.0,360.0,360.0,360.000000,360.000000,360.0,360.000000,360.000000,...,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000
mean,5.0,5.0,22050.0,5.0,110250.0,0.090699,0.143875,1.0,0.143880,0.369793,...,-53.305883,9.880133,-56.056970,8.994184,-61.324737,8.754055,-79.963797,0.400800,4596.000000,0.500000
std,0.0,0.0,0.0,0.0,0.0,0.030093,0.034150,0.0,0.034152,0.360295,...,4.408045,2.371283,4.159814,2.743630,4.941069,2.534579,0.048214,0.481910,3493.303646,0.500696
min,5.0,5.0,22050.0,5.0,110250.0,0.024499,0.061467,1.0,0.061467,-0.963850,...,-69.223130,6.320837,-69.534130,4.099804,-73.945280,3.784613,-80.000000,0.000000,1701.000000,0.000000
25%,5.0,5.0,22050.0,5.0,110250.0,0.063122,0.116380,1.0,0.116381,0.090582,...,-55.748732,8.237116,-58.745985,7.285913,-65.283685,6.389926,-80.000000,0.000000,1701.000000,0.000000
50%,5.0,5.0,22050.0,5.0,110250.0,0.096490,0.147331,1.0,0.147339,0.327049,...,-52.881535,9.102893,-56.205628,8.641234,-60.532394,9.159498,-79.985630,0.204881,2584.000000,0.500000
75%,5.0,5.0,22050.0,5.0,110250.0,0.113756,0.170039,1.0,0.170042,0.654062,...,-50.422297,11.346341,-53.530049,11.104043,-57.249506,10.619641,-79.945368,0.673471,9503.000000,1.000000
max,5.0,5.0,22050.0,5.0,110250.0,0.160985,0.221658,1.0,0.221658,1.280069,...,-42.567600,16.698685,-43.770410,15.246654,-50.570510,14.078880,-79.781870,1.980780,9503.000000,1.000000


### 2.2 Reproducir algunos audios de ejemplo

In [11]:
fake_files = glob.glob('data/procesados/*/fake/*.wav')
original_files = glob.glob('data/procesados/*/original/*.wav')

muestra = 10

if fake_files and original_files:
    print("Ejemplo de audio FAKE:")
    fake_sample = fake_files[muestra]
    print(f"Archivo: {fake_sample}")
    display(Audio(fake_sample))

    print("\nEjemplo de audio ORIGINAL:")
    original_sample = original_files[muestra]
    print(f"Archivo: {original_sample}")
    display(Audio(original_sample))
else:
    print("No se encontraron archivos de audio en las rutas esperadas.")
    print("Archivos fake encontrados:", len(fake_files))
    print("Archivos originales encontrados:", len(original_files))

Ejemplo de audio FAKE:
Archivo: data/procesados/9503/fake/9503_fake_044.wav



Ejemplo de audio ORIGINAL:
Archivo: data/procesados/9503/original/9503_original_027.wav


## 3. Preprocesamiento de Datos

### 3.1 Eliminar columnas de metadatos

In [15]:
columns_to_remove = [
    'filename', 'original_duration', 'processed_duration', 'sample_rate',
    'duration', 'samples', 'max_amplitude', 'audio_type', 'individual_id', 'path', 'File_path'
]

columns_to_remove = [col for col in columns_to_remove if col in df.columns]
df_clean = df.drop(columns=columns_to_remove)

print(f"Dimensiones: {df_clean.shape}")
print(f"Columnas eliminadas: {len(columns_to_remove)}")

numeric_columns = df_clean.select_dtypes(include=[np.number]).columns
non_numeric_columns = df_clean.select_dtypes(exclude=[np.number]).columns

if len(non_numeric_columns) > 0:
    print(f"Columnas NO numéricas: {non_numeric_columns.tolist()}")
else:
    print("Todas las columnas son numéricas")

Dimensiones: (360, 197)
Columnas eliminadas: 9
Todas las columnas son numéricas


### 3.2. Análisis de características

> Debido al gran número de características, solo se toman 20 como ejemplo

In [ ]:
representative_features = [
    # Temporales (4)
    'mean_amplitude',           # Amplitud promedio
    'std_amplitude',            # Desviación estándar de amplitud
    'zero_crossing_rate',       # Tasa de cruces por cero
    'rms',                      # RMS (Root Mean Square)

    # Envolvente (4)
    'attack_time',              # Tiempo de ataque
    'sustain_level',            # Tiempo de sostenido
    'envelope_mean',            # Media de la envolvente
    'envelope_std',             # Desviación estándar de la envolvente

    # Espectrales (4)
    'spectral_centroid_mean',   # Centroide espectral
    'spectral_rolloff_mean',    # Rolloff espectral
    'spectral_bandwidth_mean',  # Ancho de banda espectral
    'spectral_flatness_mean',   # Planitud espectral

    # Harmónicos (2)
    'harmonic_ratio',           # Ratio armónico
    'percussive_ratio',         # Ratio percusivo

    # Cromáticos (2)
    'chroma_0_mean',            # Primera banda cromática
    'chroma_5_mean',            # Sexta banda cromática

    # MFCC (2)
    'mfcc_1_mean',              # Primer coeficiente MFCC
    'mfcc_2_mean',              # Segundo coeficiente MFCC

    # Mel-Spectrogram (2)
    'mel_spec_mean',            # Promedio mel-spectrogram
    'mel_band_0_mean'           # Primera banda mel
]

#### 3.2.1. Análisis univariado

`Aqui se puede mostrar la distribución global y la distribución por clase (Que en la gráfica se noten con colores distintos la distribucion de los originales y los fake)`

> Observaciones

`Mostrar la grafica de boxplots por caracteristica`

> Observaciones

#### 3.2.2. Correlación entre atributos

> Observaciones

### 4. Modelado

> En todos los casos usar validación cruzada dependiendo del tiempo que demora, pueden ser 3, 5, 10 folds

In [17]:
target_column = 'label'

X = df_clean.drop(columns=[target_column])
y = df_clean[target_column]


print(f"Características (X): {X.shape}")
print(f"Etiquetas (y): {y.shape}")

Características (X): (360, 196)
Etiquetas (y): (360,)


In [19]:
numeric_columns = X.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()

X_numeric = X[numeric_columns]
X_scaled_numeric = scaler.fit_transform(X_numeric)

X_scaled = pd.DataFrame(X_scaled_numeric, columns=numeric_columns, index=X.index)

print(f"Dimensiones de X estandarizado: {X_scaled.shape}")
print("\nEstadísticas después de la estandarización (media ≈ 0, std ≈ 1):")
print(X_scaled[numeric_columns].describe())

Dimensiones de X estandarizado: (360, 196)

Estadísticas después de la estandarización (media ≈ 0, std ≈ 1):
       mean_amplitude  std_amplitude           rms      skewness  \
count    3.600000e+02   3.600000e+02  3.600000e+02  3.600000e+02   
mean     5.526444e-16  -3.552714e-16  3.552714e-16 -3.947460e-17   
std      1.001392e+00   1.001392e+00  1.001392e+00  1.001392e+00   
min     -2.202936e+00  -2.416472e+00 -2.416483e+00 -3.706680e+00   
25%     -9.176967e-01  -8.062321e-01 -8.063176e-01 -7.760289e-01   
50%      1.926844e-01   1.013233e-01  1.014232e-01 -1.188013e-01   
75%      7.672532e-01   7.671954e-01  7.671269e-01  7.900872e-01   
max      2.338885e+00   2.280819e+00  2.280586e+00  2.529991e+00   

           kurtosis  zero_crossing_rate   energy_mean    energy_std  \
count  3.600000e+02        3.600000e+02  3.600000e+02  3.600000e+02   
mean   1.184238e-16       -2.368476e-16  1.973730e-16 -1.825700e-16   
std    1.001392e+00        1.001392e+00  1.001392e+00  1.001392e+

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Entrenamiento: {X_train.shape[0]} muestras")
print(f"Prueba: {X_test.shape[0]} muestras")
print(f"Características: {X_train.shape[1]} features")

print("\nDistribución de clases en entrenamiento:")
print(y_train.value_counts(normalize=True))

print("\nDistribución de clases en prueba:")
print(y_test.value_counts(normalize=True))


Entrenamiento: 288 muestras
Prueba: 72 muestras
Características: 196 features

Distribución de clases en entrenamiento:
label
0    0.5
1    0.5
Name: proportion, dtype: float64

Distribución de clases en prueba:
label
0    0.5
1    0.5
Name: proportion, dtype: float64


#### 4.1. Linea base

> Describir los modelos que le tocó, usar referencias bibliográficas en formato APA (Al menos 1 referencia por modelo)

1. MLP
```
Descripción: 

Parámetros principales:
```

2. ...
3. ...


> Procesar y modelar las 196 caracteristicas sin ninguna modificación con los modelos asignados.

In [ ]:
from sklearn.linear_model import (
    LogisticRegression, RidgeClassifier, SGDClassifier, Perceptron, PassiveAggressiveClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, HistGradientBoostingClassifier,StackingClassifier
)
from sklearn.naive_bayes import (
    GaussianNB, MultinomialNB, BernoulliNB, ComplementNB
)
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
)
from sklearn.neural_network import MLPClassifier

models = [
    ('LogR', LogisticRegression()),                                      # Regresión logística
    ('Ridge', RidgeClassifier()),                                        # Regularización L2
    ('SGD', SGDClassifier()),                                            # Descenso estocástico
    ('Perceptron', Perceptron()),                                        # Neurona lineal clásica
    ('PA', PassiveAggressiveClassifier()),                               # Lineal, robusto a ruido
    ('k-NN', KNeighborsClassifier()),                                    # Vecinos más cercanos
    ('SVM', SVC()),                                                      # Kernel SVM (no lineal)
    ('LinSVM', LinearSVC()),                                             # SVM lineal (rápido con muchas features)
    ('CART', DecisionTreeClassifier()),                                  # Árbol de decisión
    ('RF', RandomForestClassifier()),                                    # Random Forest
    ('ET', ExtraTreesClassifier()),                                      # Extra Trees
    ('ADA', AdaBoostClassifier()),                                       # AdaBoost
    ('GB', GradientBoostingClassifier()),                                # Gradient Boosting clásico
    ('HGB', HistGradientBoostingClassifier()),                           # Gradient Boosting moderno (rápido)
    ('BAG', BaggingClassifier()),                                        # Bagging genérico
    ('GNB', GaussianNB()),                                               # Naive Bayes (continuo)
    ('BNB', BernoulliNB()),                                              # Naive Bayes (binario)
    ('MNB', MultinomialNB()),                                            # Naive Bayes (conteos)
    ('CNB', ComplementNB()),                                             # Variante para datos desbalanceados
    ('LDA', LinearDiscriminantAnalysis()),                               # Discriminante lineal
    ('QDA', QuadraticDiscriminantAnalysis()),                            # Discriminante cuadrático
    ('MLP', MLPClassifier()),                                            # Red neuronal
    ('SC', StackingClassifier(estimators=100))
]

### A decisión suya puede optar por agregar estos modelos (requieren instalación adicional)
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
# from sklearn.ensemble import VotingClassifier
# from sklearn.svm import NuSVC


#### 4.2. Con estandarización

> Estandarizar y luego procesar

#### 4.3. Estandarización + PCA

> Estandarizar y aplicar PCA (con 90% de varianza explicada). 

#### 4.4. Comparación de experimentos

> Crear una o varias gráficas comparativas con los experimentos realizados 

> Observaciones:

### 5. Optimización del mejor modelo

> Usar GridSearch para optimizar los parámetros del mejor modelo, con el método con los mejores resultados (si el modelo no llega al 100%)

### 6. Forecasting

> Realizar predicciones en el conjunto de prueba

> Mostrar matriz de confusión

> Conclusiones:

1.  `Sobre el preprocesamiento`
2.  `Sobre la evaluación univariada de las características`
3.  `Sobre los experimentos realizados`
4.  `Sobre la optimización del mejor modelo`
5.  `Sobre el resultado final de clasificación`

Observaciones adicionales: